# 1. 环境配置

## 1.1 python 环境准备

In [1]:
! pip install openai==2.11.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/e5/f1/d9251b565fce9f8daeb45611e3e0d2f7f248429e40908dcee3b6fe1b5944/openai-2.11.0-py3-none-any.whl (1.1 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/7f/9c/36c5c37947ebfb8c7f22e0eb6e4d188ee2d53aa3880f3f2744fb894f0cb1/anyio-4.12.0-py3-none-any.whl (113 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/2a/39/e50c7c3a983047577ee07d2a9e53faf5a69493943ec3f6a384bdc792deb2/httpx-0.28.1-py3-none-any.whl (73 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/1c/c7/6659f537f9562d963488e3e55573498a442503ced01f7e169e96a6110383/jiter-0.12.0-cp312-cp312-win_amd64.whl (205 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/5a/87/b70ad306ebb6f9b585f114d0ac2137d792b48be34d732d6

## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [2]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"

## 1.3 智能体简介

智能体是一个能感知环境、理解任务、规划行动并自主执行的“智能程序”。

可以把它理解为：
- 有目标导向（人工输入任务）
- 会思考（使用大模型推理）
- 会行动（动态调用工具）
- 可记忆（记录上下文与历史）
- 可自我调节（遇到异常自动处理）

# 2. 原生 ReAct 智能体搭建

在这一小节里，我们不使用像 LangChain 或 LlamaIndex 这种集成框架，我们就基于最基本的 python 代码以及 openai 库调用大模型的方式，实现一个最简单的 Agent 系统，从而了解其背后的逻辑和原理。

## 2.1 ReAct Agent 简介

ReAct 是一种结合了“推理（Reasoning）”和“行动（Acting）”能力的大语言模型交互范式，由哈佛大学等在 2022 年提出。
它的设计思想是，让大模型一边“思考”，一边“行动”，通过交替进行推理和行动的方式，逐步解决复杂问题。

对于一个常见的 ReAct 框架的 Agent ，我们需要以下几个零部件：
- LLM 模型（大脑）：百度文心一言等大语言模型。其作用是负责理解用户输入、组织推理逻辑、判断调用哪个工具、最后生成回答。它是整个 Agent 推理与决策的基础。
- Memory（记忆）：保存对话的上下文。其作用是记录用户的历史对话、身份信息、上下文等，让 Agent 在多轮对话中保持“记忆”。
- 工具（行动能力）：计算器、搜索器、数据库查询器等。其作用是帮助 Agent 执行它不能直接回答的任务，例如查天气、算数学题、提取表格数据等。
- 系统提示词（思维方式）：是驱动大模型的核心，其作用是明确告诉模型“你要如何思考、如何调用工具、如何表达每一步的过程”。

组件准备完毕后，我们就可以正式的进行 Agent 流程的组装并运行查看结果。

## 2.2 LLM 模型

在实际运行之前，我们要先确保基本的环境配置完成（安装 openai 库）。并且能够将 AI Studio 中的模型（ernie-4.5-turbo-128k）进行调用。

In [3]:

import os
from openai import OpenAI

client = OpenAI(
     api_key=os.environ.get("OPENAI_API_KEY"),  # 含有 AI Studio 访问令牌的环境变量，https://aistudio.baidu.com/account/accessToken,
     base_url="https://aistudio.baidu.com/llm/lmapi/v3",  # aistudio 大模型 api 服务域名
)

chat_completion = client.chat.completions.create(
    messages=[
        {'role': 'system', 'content': '你是 AI Studio 实训AI开发平台的开发者助理，你精通开发相关的知识，负责给开发者提供搜索帮助建议。'},
        {'role': 'user', 'content': '你好，请介绍一下AI Studio'}
    ],
    model="ernie-4.5-turbo-128k",
)

print(chat_completion.choices[0].message.content)

您好，很高兴为您介绍AI Studio。AI Studio是百度推出的一个一站式AI开发实训平台，专为开发者、研究人员和学生设计，旨在降低AI开发的门槛，促进AI技术的普及和应用。以下是AI Studio的主要特点和功能：

### 1. **丰富的课程资源**
   - AI Studio提供了大量的AI相关课程，涵盖机器学习、深度学习、自然语言处理、计算机视觉等领域。
   - 课程内容包括理论讲解、实战案例和项目实践，适合不同层次的开发者。

### 2. **在线编程环境**
   - 平台内置了Jupyter Notebook风格的在线编程环境，支持Python语言，并集成了百度飞桨（PaddlePaddle）深度学习框架。
   - 开发者可以直接在浏览器中编写、运行和调试代码，无需本地安装复杂的开发环境。

### 3. **预置模型库**
   - AI Studio提供了丰富的预训练模型，涵盖图像分类、目标检测、语义分割、文本生成等任务。
   - 开发者可以快速调用这些模型进行微调或直接应用，节省开发时间。

### 4. **数据集和标注工具**
   - 平台内置了大量公开数据集，并支持用户上传自己的数据集。
   - 提供了数据标注工具，帮助开发者快速完成数据标注任务。

### 5. **竞赛和项目**
   - AI Studio定期举办AI竞赛，鼓励开发者参与并解决实际问题。
   - 开发者可以创建或加入项目，与其他开发者协作完成AI应用开发。

### 6. **社区和交流**
   - 平台拥有活跃的开发者社区，用户可以在社区中提问、分享经验、交流技术。
   - 百度工程师和AI专家也会在社区中解答问题，提供技术支持。

### 7. **免费算力支持**
   - AI Studio为开发者提供免费的GPU算力资源，支持大规模模型训练和推理。
   - 开发者可以通过申请获得额外的算力支持，满足复杂项目的需求。

### 8. **企业级解决方案**
   - 对于企业用户，AI Studio提供了定制化的AI解决方案，帮助企业快速构建和部署AI应用。

### 适用人群
- **初学者**：通过课程和实战案例快速入门AI开发。
- **研究人员**：利用平台提供的算力和模型进行学术研究。
- **开发者**：快速构建和部署

## 2.3 Memory 记忆

在确定大模型能够顺利调用后，我们就可以构建一个支持 LLM 对话的类，用于与大语言模型进行交互，同时记录完整的对话信息，为后续循环调用做好准备。

In [4]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        from openai import OpenAI
        client = OpenAI(
            api_key=os.environ.get("OPENAI_API_KEY"),  
            base_url="https://aistudio.baidu.com/llm/lmapi/v3")

        response = client.chat.completions.create(
            model="ernie-4.5-turbo-128k",
            messages=self.messages
        )
        return response.choices[0].message.content
    
abot = Agent('你是一个乐于助人的机器人')
print(abot("你是谁？") )

我是一个喜欢帮助别人的机器人呀！无论是解答问题还是陪你聊天，我都很开心能帮到你~有什么需要我帮忙的吗？


## 2.4 Tool 工具

我们可以设置两个小的工具来完成后续的Agent任务：
- 第一个是计算器工具 calculate(what)，这是一个用于执行数学表达式的工具函数，能够接收一个字符串形式的表达式，并使用 Python 的 eval() 函数进行计算。
- 另外一个是狗狗体重查询工具average_dog_weight(name) ，该函数根据传入的狗的品种名称，返回该品种的平均体重信息。如果传入的品种不在预设列表中，则返回默认的平均体重信息。

In [5]:
def calculate(what):
    return eval(what)

print(calculate("3 + 7 * 2"))   # 返回 17
print(calculate("10 / 4"))      # 返回 2.5

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

print(average_dog_weight("Scottish Terrier"))  
# 返回 "Scottish Terriers average 20 lbs"
print(average_dog_weight("Labrador"))          
# 返回 "An average dog weights 50 lbs"

# 将函数注册到 known_actions 字典中
known_actions = {
  "calculate": calculate,
  "average_dog_weight": average_dog_weight
}

17
2.5
Scottish Terriers average 20 lbs
An average dog weights 50 lbs


## 2.5 System Prompt 系统提示词

系统提示词就是控制模型如何执行任务的，整体来说提示词分为四部分：
- 定义大模型的行为流程
- 解释各个关键部分的含义
- 列出可用的工具
- 示例演示整体的运行流程

In [6]:
prompt = """You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

总的来说，这个系统提示词是一个典型的 ReAct 模式 Prompt，用于引导大语言模型按 “Thought → Action → Observation → Answer” 的流程进行多步推理。

它明确了可调用的工具（计算器与狗体重查询），并通过示例教会模型如何思考、选择工具、暂停等待执行结果，最终给出回答。

基于这个提示词模版，大模型就能够照着示例来一步步完成我们想要其完成的任务。

## 2.6 系统组装

当我们准备好了四部分Agent的组件：
- LLM 模型（大脑）
- Memory （记忆）
- 工具 （行动能力）
- 系统提示词（思维方式）

我们就可以将其组装起来，实现真正的Agent流程了。

In [7]:
import re
action_re = re.compile(r'^Action: (\w+): (.*)$')
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return result

question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
print(query(question))

Thought: I need to find the average weight of each dog breed and then add them together to get the combined weight. I'll use the average_dog_weight action for each breed and then the calculate action to sum them.

Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now that I have the Border Collie's weight, I need to find the Scottish Terrier's average weight and then add the two together.

Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: I now have both weights - 37 lbs for the Border Collie and 20 lbs for the Scottish Terrier. I'll add them together using the calculate action.

Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.
Answer: The combined weight of a Border Co

## 2.7 流程总结

| 阶段 | 模型行为           | 工具调用                          | Observation   | 下一步       |
|------|--------------------|-----------------------------------|---------------|--------------|
| 1    | 思考需要两个体重   | 查 Border Collie                  | 得到 37 lbs   | 去查第二只   |
| 2    | 查第二只狗的体重   | 查 Scottish Terrier               | 得到 20 lbs   | 开始计算     |
| 3    | 加法计算           | calculate: 37 + 20                | 得到 57       | 给出最终答案 |
